# XGBoost Nowcasting — Notebook (cells tách riêng)
Dự báo mưa 60 phút tới từ dữ liệu IoT 5 phút. Không lưu model, chỉ in metrics.

**Yêu cầu dữ liệu** đặt trong `data/`:
- `sensor_raw_60d.csv`
- `labels_rain_60d.csv`
- `irrigation_events_60d.csv` (tùy chọn)


## 0) Cài thư viện (chạy một lần nếu chưa có)
```bash
!pip install pandas numpy scikit-learn xgboost
```

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    precision_recall_fscore_support,
    classification_report, confusion_matrix
)

## 1) Load dữ liệu & ghép nhãn

In [3]:
RAW_CSV = "../data/sensor_raw_60d.csv"
LBL_CSV = "../data/labels_rain_60d_fixed.csv"
IRRIG_CSV = "../data/irrigation_events_60d.csv"  # optional

raw = pd.read_csv(RAW_CSV, parse_dates=["ts"]).sort_values(["device_id","ts"]).reset_index(drop=True)
lbl = pd.read_csv(LBL_CSV, parse_dates=["ts"])  # contains rain_next_60

df = raw.merge(lbl[["ts","device_id","rain_next_60"]], on=["ts","device_id"], how="inner")
print(f"Loaded: raw={raw.shape}, labels={lbl.shape}, merged={df.shape}")

def enrich_irrigation(df: pd.DataFrame, irrig_path: str) -> pd.DataFrame:
    """Đánh dấu các mốc đang tưới và tổng phút tưới 3h/6h. Nếu không có file -> điền 0."""
    try:
        irrig = pd.read_csv(irrig_path, parse_dates=["start_ts","end_ts"])
    except Exception:
        df["is_irrigating_now"], df["irrig_total_min_last_3h"], df["irrig_total_min_last_6h"] = 0, 0.0, 0.0
        return df

    df = df.copy()
    df["is_irrigating_now"] = 0
    for _, ev in irrig.iterrows():
        mask = (
            (df["device_id"]==ev["device_id"]) &
            (df["ts"]>=ev["start_ts"]) & (df["ts"]<=ev["end_ts"]) 
        )
        df.loc[mask, "is_irrigating_now"] = 1

    df = df.sort_values(["device_id","ts"]).reset_index(drop=True)
    df["irrig_min_5"] = df["is_irrigating_now"] * 5
    df["irrig_total_min_last_3h"] = (
        df.groupby("device_id")["irrig_min_5"].rolling(36, min_periods=1).sum().reset_index(level=0, drop=True)
    )
    df["irrig_total_min_last_6h"] = (
        df.groupby("device_id")["irrig_min_5"].rolling(72, min_periods=1).sum().reset_index(level=0, drop=True)
    )
    return df

df = enrich_irrigation(df, IRRIG_CSV)

Loaded: raw=(17280, 7), labels=(17280, 4), merged=(17280, 8)


## 2) Tạo đặc trưng (feature engineering)

In [4]:
# === Feature Engineering (Pro) — 15/30/60/120', deltas dài, rolling, time cycles ===
import numpy as np
import pandas as pd

df = df.sort_values(["device_id","ts"]).reset_index(drop=True)

def add_feats_pro(g: pd.DataFrame) -> pd.DataFrame:
    g = g.copy()

    base_cols = ["temp_c","rh_pct","pressure_hpa","soil_moist_pct"]
    # nếu có dữ liệu thời tiết ngoài, thêm vào pipeline
    ext_cols = [c for c in ["api_rain_prob_60","api_rain_mm_60","cloud_pct","wind_mps"] if c in g.columns]
    all_cols = base_cols + ext_cols

    # ---- Lags (15', 30', 60', 120')
    for col in all_cols:
        for k in (3, 6, 12, 24):
            g[f"{col}_lag{k}"] = g[col].shift(k)

    # ---- Rolling mean/std cho 30', 60', 120'
    for col in all_cols:
        for w in (6, 12, 24):
            g[f"{col}_mean{w}"] = g[col].rolling(w).mean()
            g[f"{col}_std{w}"]  = g[col].rolling(w).std()

    # ---- Deltas dài (30', 60')
    for col in all_cols:
        g[f"{col}_d30"] = g[col] - g[col].shift(6)
        g[f"{col}_d60"] = g[col] - g[col].shift(12)

    # ---- Tín hiệu mưa quá khứ
    if "rain_mm_5min" in g.columns:
        g["rain_in_last_15m"] = g["rain_mm_5min"].rolling(3).sum().gt(0).astype(int)
        g["rain_last_30m"]    = g["rain_mm_5min"].rolling(6).sum()
        g["rain_last_60m"]    = g["rain_mm_5min"].rolling(12).sum()

    # ---- Time features (chu kỳ ngày/tuần)
    g["hour_of_day"] = g["ts"].dt.hour
    g["day_of_week"] = g["ts"].dt.dayofweek
    g["month"]       = g["ts"].dt.month

    g["hod_sin"] = np.sin(2*np.pi*g["hour_of_day"]/24.0)
    g["hod_cos"] = np.cos(2*np.pi*g["hour_of_day"]/24.0)
    g["dow_sin"] = np.sin(2*np.pi*(g["day_of_week"]+1)/7.0)
    g["dow_cos"] = np.cos(2*np.pi*(g["day_of_week"]+1)/7.0)

    # ---- Một vài tương tác nhẹ (hữu ích cho mưa đối lưu)
    if {"pressure_hpa_d30","rh_pct_d30"}.issubset(g.columns):
        g["rh_over_pressure_d30"] = g["rh_pct_d30"] / (g["pressure_hpa_d30"].abs() + 1e-6)
    if {"pressure_hpa_d60","rh_pct_d60"}.issubset(g.columns):
        g["rh_over_pressure_d60"] = g["rh_pct_d60"] / (g["pressure_hpa_d60"].abs() + 1e-6)

    return g

df = df.groupby("device_id", group_keys=False).apply(add_feats_pro)

# Bỏ các hàng đầu chuỗi (do lag/rolling) + reset index
df = df.dropna().reset_index(drop=True)

# ---- Danh sách FEATURES động: gồm base, external nếu có, lags/rolling/deltas/time/irrigation
base_feats = [
    # gốc
    "temp_c","rh_pct","pressure_hpa","soil_moist_pct","rain_mm_5min",
    # mưa quá khứ
    "rain_in_last_15m","rain_last_30m","rain_last_60m",
    # time encoding
    "hour_of_day","day_of_week","month","hod_sin","hod_cos","dow_sin","dow_cos",
    # tưới (nếu đã enrich)
    "is_irrigating_now","irrig_total_min_last_3h","irrig_total_min_last_6h",
]

# thêm external nếu có
ext_feats = [c for c in ["api_rain_prob_60","api_rain_mm_60","cloud_pct","wind_mps"] if c in df.columns]

# sinh tên lags/rolling/deltas cho tất cả cột gốc + external
def expand(names):
    out = []
    for col in names:
        out += [f"{col}_lag{k}" for k in (3,6,12,24)]
        out += [f"{col}_mean{w}" for w in (6,12,24)]
        out += [f"{col}_std{w}"  for w in (6,12,24)]
        out += [f"{col}_d30", f"{col}_d60"]
    return out

dyn_from_base = expand(["temp_c","rh_pct","pressure_hpa","soil_moist_pct"])
dyn_from_ext  = expand(ext_feats) if ext_feats else []

# tương tác
inter_feats = [c for c in ["rh_over_pressure_d30","rh_over_pressure_d60"] if c in df.columns]

FEATURES = []
for c in base_feats + ext_feats + dyn_from_base + dyn_from_ext + inter_feats:
    if c in df.columns:
        FEATURES.append(c)

# tạo ma trận
X = df[FEATURES].astype("float32").values
y = df["rain_next_60"].astype(int).values   # dùng nhãn đã FIX/REBUILD
print(f"Features: {len(FEATURES)} cols | X: {X.shape}  y: {y.shape}")


Features: 68 cols | X: (17256, 68)  y: (17256,)


C:\Users\Vu The Van\AppData\Local\Temp\ipykernel_10708\3805558836.py:55: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("device_id", group_keys=False).apply(add_feats_pro)


In [5]:
# 1) In danh sách FEATURES để xem có cột rò rỉ tương lai không
print("External features in use?",
      any(c in FEATURES for c in ["api_rain_prob_60","api_rain_mm_60"]))
print("Current-rain features?",
      any(c in FEATURES for c in ["rain_mm_5min","rain_last_30m","rain_last_60m"]))
print("Label-like columns accidentally in X?",
      any(c in FEATURES for c in ["rain_next_60","rain_amount_next_60_mm","rain_next_60_rebuilt","rain_amount_next_60_mm_rebuilt"]))


External features in use? False
Current-rain features? True
Label-like columns accidentally in X? False


## 3) Chia train/val + Train XGBoost (early stopping)

In [ ]:
# === Train XGBoost (opt params) + Early Stopping + Threshold Search (F1/Recall) — SAFE ===
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_fscore_support,
    accuracy_score, precision_recall_curve
)

# 1) Split giữ thứ tự thời gian
pos, neg = (y == 1).sum(), (y == 0).sum()
scale_pos_weight = float(neg) / max(1.0, float(pos))

Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.15, shuffle=False)

dtrain = xgb.DMatrix(Xtr, label=ytr)
dvalid = xgb.DMatrix(Xte, label=yte)

# 2) Tham số khởi điểm tốt cho nowcasting mưa (có thể tinh chỉnh thêm)
params = {
    "objective": "binary:logistic",
    "eval_metric": ["logloss", "auc"],
    "eta": 0.03,
    "max_depth": 6,
    "min_child_weight": 3,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "lambda": 2.0,
    "alpha": 0.0,
    "scale_pos_weight": scale_pos_weight,
    "tree_method": "hist",  # GPU: "gpu_hist"
}

bst = xgb.train(
    params=params,
    dtrain=dtrain,
    num_boost_round=1200,
    evals=[(dtrain, "train"), (dvalid, "valid")],
    early_stopping_rounds=100,
    verbose_eval=False,
)

# 3) Dự đoán proba trên tập validation theo best_iter

def _predict_proba_booster(booster, Xmat):
    dm = xgb.DMatrix(Xmat)
    best_ntree_limit = getattr(booster, "best_ntree_limit", None)
    best_iteration = getattr(booster, "best_iteration", None)
    if best_ntree_limit is not None:
        p = booster.predict(dm, ntree_limit=best_ntree_limit)
    elif best_iteration is not None:
        p = booster.predict(dm, iteration_range=(0, best_iteration + 1))
    else:
        p = booster.predict(dm)
    return np.asarray(p).reshape(-1)

proba = _predict_proba_booster(bst, Xte)

# 4) Metric @0.50 (tham khảo)
pred_05 = (proba >= 0.5).astype(int)
auc = roc_auc_score(yte, proba)
pr_auc = average_precision_score(yte, proba)
prec05, rec05, f105, _ = precision_recall_fscore_support(yte, pred_05, average="binary")
acc05 = accuracy_score(yte, pred_05)
print(f"AUC-ROC: {auc:.4f} | PR-AUC: {pr_auc:.4f}")
print(f"@0.50  Acc: {acc05:.4f}  Prec: {prec05:.4f}  Rec: {rec05:.4f}  F1: {f105:.4f}")

# 5) Quét ngưỡng: (a) F1-best, (b) Recall-first an toàn
prec, rec, thr = precision_recall_curve(yte, proba)

# F1-best (giữ nguyên)
f1s = 2 * prec * rec / (prec + rec + 1e-12)
ix_f1 = int(np.argmax(f1s))
thr_f1 = float(thr[ix_f1]) if ix_f1 < len(thr) else 0.5

# Recall-first (safe): align độ dài, kiểm tra rỗng
valid = np.isfinite(prec[:-1]) & np.isfinite(rec[:-1])
prec_cut = prec[:-1][valid]
rec_cut  = rec[:-1][valid]
thr_cut  = thr[valid]

target_recall = 0.60  # CHỈNH theo mục tiêu
mask = rec_cut >= target_recall
if np.any(mask):
    cand_idx  = np.where(mask)[0]
    cand_thr  = thr_cut[cand_idx]
    cand_prec = prec_cut[cand_idx]
    j_local   = int(np.argmax(cand_prec))
    thr_rec   = float(cand_thr[j_local])
    print(f"Recall≥{target_recall:.2f} candidate thr: {thr_rec:.3f} (prec={cand_prec[j_local]:.4f}, rec={rec_cut[cand_idx][j_local]:.4f})")
else:
    thr_rec = None
    print(f"Không có ngưỡng đạt Recall ≥ {target_recall:.2f} trên tập validation.")

# 6) Wrapper: dùng ngưỡng mặc định ƯU TIÊN RECALL -> F1 -> 0.5
class XGBBoosterWithThreshold:
    def __init__(self, booster, threshold):
        self._booster = booster
        self.threshold = float(threshold)
    def predict_proba(self, Xmat):
        p1 = _predict_proba_booster(self._booster, Xmat)
        return np.c_[1.0 - p1, p1]
    def predict(self, Xmat, threshold=None):
        th = self.threshold if threshold is None else float(threshold)
        p1 = _predict_proba_booster(self._booster, Xmat)
        return (p1 >= th).astype(int)
    def get_booster(self):
        return self._booster

default_thr = (thr_rec if (thr_rec is not None) else (thr_f1 if thr_f1 is not None else 0.5))
model = XGBBoosterWithThreshold(bst, default_thr)
print(f"Model sẵn sàng. NGƯỠNG MẶC ĐỊNH: {model.threshold:.3f}")

# import json, joblib, os
# os.makedirs("models", exist_ok=True)

# # 1) Lưu classifier 60'
# joblib.dump(model, "models/xgb_nowcast.pkl")   # model có .predict_proba() và thuộc tính .threshold

# # 2) (tuỳ chọn) lưu regressor lượng mưa 60' nếu bạn đã train
# # joblib.dump(reg_model, "models/xgb_amount.pkl")

# # 3) Lưu metadata (FEATURES + threshold)
# meta = {
#     "features": FEATURES,
#     "threshold_default": float(getattr(model, "threshold", 0.5)),
#     "note": "XGBoost nowcasting 60' | threshold ưu tiên Recall nếu đã cấu hình"
# }
# with open("models/metadata.json", "w", encoding="utf-8") as f:
#     json.dump(meta, f, ensure_ascii=False, indent=2)

AUC-ROC: 0.9624 | PR-AUC: 0.9264
@0.50  Acc: 0.9714  Prec: 0.9290  Rec: 0.9290  F1: 0.9290
Recall≥0.60 candidate thr: 0.614 (prec=0.9304, rec=0.9232)
Model sẵn sàng. NGƯỠNG MẶC ĐỊNH: 0.614


## 4) Đánh giá: AUC/PR-AUC/Precision/Recall/F1 + báo cáo

In [34]:
# === Evaluation (robust) ===
import numpy as np
from sklearn.metrics import (
    roc_auc_score, average_precision_score, precision_recall_fscore_support,
    precision_recall_curve, classification_report, confusion_matrix, accuracy_score
)

# ---- 1) Lấy proba từ model wrapper hoặc booster gốc
proba = None

if 'model' in globals() and hasattr(model, 'predict_proba'):
    proba = model.predict_proba(Xte)[:, 1]

if proba is None:
    import xgboost as xgb
    if 'bst' in globals():
        dte = xgb.DMatrix(Xte)
        best_ntree_limit = getattr(bst, 'best_ntree_limit', None)
        best_iteration   = getattr(bst, 'best_iteration', None)
        if best_ntree_limit is not None:
            proba = bst.predict(dte, ntree_limit=best_ntree_limit)
        elif best_iteration is not None:
            proba = bst.predict(dte, iteration_range=(0, best_iteration + 1))
        else:
            proba = bst.predict(dte)
    else:
        raise RuntimeError("Chưa có `model` hay `bst`. Hãy chạy cell Train trước.")

proba = np.asarray(proba).reshape(-1)

# ---- 2) AUC & PR-AUC
auc = roc_auc_score(yte, proba)
pr_auc = average_precision_score(yte, proba)
print(f"AUC-ROC: {auc:.4f} | PR-AUC: {pr_auc:.4f}")

# ---- 3) các ngưỡng cần đánh giá
def metrics_at(y_true, p, thr):
    pred = (p >= thr).astype(int)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, pred, average="binary", zero_division=0)
    acc = accuracy_score(y_true, pred)
    tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
    return {
        "threshold": float(thr),
        "accuracy": float(acc),
        "precision": float(prec),
        "recall": float(rec),
        "f1": float(f1),
        "tp": int(tp), "fp": int(fp), "tn": int(tn), "fn": int(fn)
    }

# 3a) @0.50
rows = [metrics_at(yte, proba, 0.5)]

# 3b) @model.threshold (nếu có)
chosen_thr = None
if 'model' in globals() and hasattr(model, 'threshold'):
    rows.append(metrics_at(yte, proba, float(model.threshold)))
    chosen_thr = float(model.threshold)

# 3c) @best-F1 (quét theo PR curve)
prec, rec, thr = precision_recall_curve(yte, proba)
f1s = 2*prec*rec/(prec+rec+1e-12)
ix = f1s.argmax()
thr_f1 = float(thr[ix]) if ix < len(thr) else 0.5
rows.append(metrics_at(yte, proba, thr_f1))

# ---- 4) In bảng tóm tắt
import pandas as pd
eval_df = pd.DataFrame(rows).drop_duplicates(subset=["threshold"]).sort_values("threshold").reset_index(drop=True)
print("\n=== Summary by threshold ===")
display(eval_df.style.format({"threshold": "{:.3f}", "accuracy": "{:.4f}", "precision": "{:.4f}", "recall":"{:.4f}", "f1":"{:.4f}"}))

# ---- 5) Chọn ngưỡng để báo cáo chi tiết
# Ưu tiên: model.threshold nếu có, nếu không dùng best-F1
if chosen_thr is None:
    chosen_thr = thr_f1

pred = (proba >= chosen_thr).astype(int)
print(f"\n=== Detailed report @threshold={chosen_thr:.3f} ===\n")
print(classification_report(yte, pred, digits=4))
print("--- Confusion Matrix (y_true rows x y_pred cols) ---")
print(confusion_matrix(yte, pred))


AUC-ROC: 0.9624 | PR-AUC: 0.9264

=== Summary by threshold ===


,threshold,accuracy,precision,recall,f1,tp,fp,tn,fn
0,0.500,0.9714,0.9290,0.9290,0.9290,484,37,2031,37
1,0.611,0.9714,0.9290,0.9290,0.9290,484,37,2031,37
2,0.614,0.9706,0.9304,0.9232,0.9268,481,36,2032,40



=== Detailed report @threshold=0.614 ===

              precision    recall  f1-score   support

           0     0.9807    0.9826    0.9816      2068
           1     0.9304    0.9232    0.9268       521

    accuracy                         0.9706      2589
   macro avg     0.9555    0.9529    0.9542      2589
weighted avg     0.9706    0.9706    0.9706      2589

--- Confusion Matrix (y_true rows x y_pred cols) ---
[[2032   36]
 [  40  481]]


## 5) Feature importance (gain)

In [26]:
importances = model.get_booster().get_score(importance_type='gain')
imp_list = [(f, importances.get(f, 0.0)) for f in FEATURES]
imp_df = pd.DataFrame(imp_list, columns=["feature","gain"]).sort_values("gain", ascending=False)
imp_df.reset_index(drop=True, inplace=True)
imp_df.head(20)

,feature,gain
0,temp_c,0.0
1,rh_pct,0.0
2,pressure_hpa,0.0
3,soil_moist_pct,0.0
4,rain_mm_5min,0.0
5,pressure_delta15,0.0
6,rh_delta15,0.0
7,temp_delta15,0.0
8,temp_c_lag15,0.0
9,rh_pct_lag15,0.0


In [28]:
import numpy as np, pandas as pd
print("Prevalence y=1:", y.mean().round(4), f"({y.sum()}/{len(y)})")
print("Val prevalence:", yte.mean().round(4), f"({yte.sum()}/{len(yte)})")
# Baseline đoán toàn 0
from sklearn.metrics import f1_score
print("Baseline F1(all-0):", f1_score(yte, np.zeros_like(yte)))


Prevalence y=1: 0.2324 (4014/17275)
Val prevalence: 0.201 (521/2592)
Baseline F1(all-0): 0.0


In [29]:
tmp = pd.DataFrame({
    "rain_next_60": y,
    "rain_in_last_15m": df.loc[:, "rain_in_last_15m"].values
})
print(tmp.corr().round(3))
# Nếu hệ số ~0 hoặc âm -> nghi lệch thời gian/merge sai.
# Thử dịch nhãn +1 step xem có tăng tương quan không:
y_shifted = np.roll(y, -3)  # lệch 15'
print("Corr with shifted y:", np.corrcoef(y_shifted[:-3], tmp["rain_in_last_15m"].values[:-3])[0,1].round(3))


                  rain_next_60  rain_in_last_15m
rain_next_60             1.000             0.511
rain_in_last_15m         0.511             1.000
Corr with shifted y: 0.513


In [17]:
dup = df.duplicated(subset=["device_id","ts"]).sum()
print("Duplicates (device_id, ts):", dup)


Duplicates (device_id, ts): 0


In [8]:
!pip install flask python-dotenv pandas numpy joblib xgboost


  Using cached flask-3.1.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached itsdangerous-2.2.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached werkzeug-3.1.3-py3-none-any.whl.metadata (3.7 kB)
Using cached flask-3.1.2-py3-none-any.whl (103 kB)
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
Using cached blinker-1.9.0-py3-none-any.whl (8.5 kB)
Using cached itsdangerous-2.2.0-py3-none-any.whl (16 kB)
Using cached werkzeug-3.1.3-py3-none-any.whl (224 kB)

   ---------------------------------------- 0/6 [werkzeug]
   -------------------------- ------------- 4/6 [blinker]
   ---------------------------------------- 6/6 [flask]

